In [1]:
# F_19, p=19, дивизор D = P_0 + ... + P_18, дивизор G = 14*P_infinity
# f(x) = (x^7 + 8*x^5 + 17*x^4 + 15*x^3 + 13*x^2) / (x^2 + 6*x + 9)


In [2]:
p = 19
delta = 14
# ------ 1.  Построить функциональное поле F 
F.<x> = FunctionField(GF(p))
print('(1) Построим функциональное поле F:', F, end='\n\n')

# ------ 2. Определить род поля
print('(2) Определим род поля:', F.genus(), end='\n\n')

# ------ 3. Определить поле констант
print('(3) Определим поле констант:', F.constant_field(), end='\n\n')

# ------ 4. Посчитать дивизор функции f
f = (x^7 + 8*x^5 + 17*x^4 + 15*x^3 + 13*x^2) / (x^2 + 6*x + 9) 
print('(4) Посчитаем дивизор функции f:\n', f.divisor(), end='\n\n')

(1) Построим функциональное поле F: Rational function field in x over Finite Field of size 19

(2) Определим род поля: 0

(3) Определим поле констант: Finite Field of size 19

(4) Посчитаем дивизор функции f:
 -5*Place (1/x)
 + 2*Place (x)
 - 2*Place (x + 3)
 + Place (x + 4)
 + Place (x + 5)
 + 2*Place (x + 7)
 + Place (x + 15)



In [3]:
# ------ 5. Построить базис пространства Римана-Роха L(G) = L(delta*P(inf))

G = delta*F.place_infinite()            # G = delta * точку бесконечность из F
L_G = G.basis_function_space()          # сам базис пр-ва Римана-Роха
L_G.reverse()                           
print('(5) Построим базис пространства Римана-Роха L(G):\n', L_G, end='\n\n')

# ------ 6. Определить размерность пространства Римана-Роха L(G)

k = G.dimension()
print('(6) Определим размерность пространства Римана-Роха L(G):', k, end='\n\n')

# ------ 7. Определить параметры кода

from sage.rings.function_field.divisor import divisor 
# теперь можно именно построить дивизор

tochki = {place: 1 for place in F.places_finite()} 
# берём все точки, кроме P_inf, у всех коэф-т перед точкой равен 1

D = divisor(F, tochki).support() 
# нужны носители, т.е. все точки по отдельности, а не в виде суммы 

D = [D[0]] + D[len(D):0:-1] 

n = len(tochki)                         # длина кода, кол-во точек
d = n - G.degree()
print(d == n - k + 1)
print(f'(7) Определим параметры кода: n = {n}, d = {d}, t = {(d - 1) // 2}, k = {k}', end='\n\n')

# ------ 8. Выбрать произвольный вектор соответствующей длины(сообщение),закодировать его, внести в результат ошибку

M = Matrix([[L_G[i].evaluate(point) for point in D] for i in range(k)]) 
# k штук строчек в матрице; считаем значение каждого эл-та базиса в точке
print("Матрица M:\n")
print(M, end = '\n\n')

u = vector([randint(1, p) for i in range(k)])       # произвольный вектор
print('(8) Выберем произвольный вектор соответствующей длины(сообщение):\n', u, end='\n\n')

c = u*M
print('Закодируем выбранный произвольный вектор:\n', c, end='\n\n')

e = vector([0 for i in range(n)])
e[randint(0, n-1)] = randint(1, p-1)
# сначала составили нулевой вектор, затем в произвольное место добавляем произвольную ошибку
print(f'Вектор ошибки e = {e}', end ='\n\n' )

c1 = c + e
# внесли ошибку 

print('Внесём в результат ошибку:\n', c1, end='\n\n')
      
# ------ 9. Декодировать кодовое слово

t = 1
# t = (x - ...)(x - ...) .... (x - ...)
for point in D:
    t *= point.local_uniformizer()  # локальное представление у точек, т.е. (x - ...), перемножаем их

# (w) = -(t)+(-2P_inf)
divisor_w = -t.divisor() - 2*F.place_infinite()
# (w) = -D + (какое-то кол-во точек P_inf)

# G~ = D - (кол-во точек P_inf в G) + (w), получится какое-то кол-во точек P_inf
G_s_volnoi = divisor(F, tochki) - G + divisor_w

# для удобства сделаем так, чтобы G1 = G~
G1 = G_s_volnoi

basis_G1 = G1.basis_function_space()
basis_G1.reverse()
basis_G_s_volnoi = G_s_volnoi.basis_function_space()

# затем нужно решать систему, [c, f_lambda * g_ro]*x_lambda = 0
# необходимо посчитать все [ ... ]

iz_sistemi = []
# в этом списке будем хранить значения [...], т.к. ниже они понадобятся

# берём базисный эл-т, считаем сумму для него
for basis_element in basis_G1:
    res = 0
    for i in range(n):
        res += c1[i]*basis_element.evaluate(D[i]) 
        # из c1 берём i-ый эл-т, умножаем его на базисный эл-т, посчитанный в т. D[i]
    iz_sistemi.append(res)

# из списка хранимых значений "iz_sistemi" нужно выбрать такую переменную, чтобы перед ней не стоял 0, тогда 
# её можно будет посчитать

x1 = var('x1')     # завели переменную 

i = 0
# нужно найти такой индекс, чтобы для него значение iz_sistemi было не равно 0
while (iz_sistemi[i] == 0) and i < n:
    i += 1

# когда нашли такой индекс, находим значение переменной
temp = int(sum(iz_sistemi) - iz_sistemi[i])
x1 = solve_mod([x1*int(iz_sistemi[i]) + temp == 0], p)[0][0]

# составляем функцию локатора ошибок, 
function_locator = 0
for j, basis_element in enumerate(basis_G1):   # берём соответствующие значения из базиса G1
    if j != i:
        function_locator += basis_element
    else:
        function_locator += x1 * basis_element   # работа с найденным выше значением переменной

# нужно посмотреть, для каких точек ф-ия локатора ошибок = 0 
points_f = []        
for point in D:
    if function_locator.evaluate(point) == 0:
        points_f.append(point)
#print("points:", points_f)   

# заводим другую переменную
z = var('z')

# вторая система, в источнике обозначали её (**), ищем значения ошибок и убираем их
for point in points_f:
    error_or_not = solve_mod([int(basis_G1[j].evaluate(point))*z == int(iz_sistemi[j]) for j in range(len(iz_sistemi))], p)
    if error_or_not != []:
        e1 = vector([0 for i in range(n)])
        e1[D.index(point)] = error_or_not[0][0]
        c2 = c1 - e1

print('(9) Декодируем кодовое слово:\n', c2, end='\n\n')

print('Проверяем:', c-c2, end = '\n\n')
print('Совпадает ли декодированное слово с исходным?', c2 == c)


(5) Построим базис пространства Римана-Роха L(G):
 [1, x, x^2, x^3, x^4, x^5, x^6, x^7, x^8, x^9, x^10, x^11, x^12, x^13, x^14]

(6) Определим размерность пространства Римана-Роха L(G): 15

True
(7) Определим параметры кода: n = 19, d = 5, t = 2, k = 15

Матрица M:

[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 0  1  4  9 16  6 17 11  7  5  5  7 11 17  6 16  9  4  1]
[ 0  1  8  8  7 11  7  1 18  7 12  1 18 12  8 12 11 11 18]
[ 0  1 16  5  9 17  4  7 11  6  6 11  7  4 17  9  5 16  1]
[ 0  1 13 15 17  9  5 11 12 16  3  7  8 14 10  2  4  6 18]
[ 0  1  7  7 11  7 11  1  1 11 11  1  1 11  7 11  7  7  1]
[ 0  1 14  2  6 16  9  7  8  4 15 11 12 10  3 13 17  5 18]
[ 0  1  9  6  5  4 16 11  7 17 17  7 11 16  4  5  6  9  1]
[ 0  1 18 18  1  1  1  1 18  1 18  1 18 18 18 18  1  1 18]
[ 0  1 17 16  4  5  6  7 11  9  9 11  7  6  5  4 16 17  1]
[ 0  1 15 10 16  6 17 11 12  5 14  7  8  2 13  3  9  4 18]
[ 0  1 11 11  7 11  7  1 